# Third fine-tuning with a 0.5B parameter model, using `DataParallel`

My goal is to fine-tune meta-llama/Meta-Llama-3-8B on timdettmers/openassistant-guanaco.  This notebook reworks `second-0.5b-fine-tune.ipynb` so that it can use the `DataParallel` wrapper to tune over multiple GPUs.  It also strips out a lot of the investigatory stuff from that notebook.

## The dataset


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
dataset_source = "timdettmers/openassistant-guanaco"

In [3]:
from datasets import load_dataset

dataset = load_dataset(dataset_source)

Repo card metadata block was not found. Setting CardData to empty.


## The model


In [4]:
base_model = "Qwen/Qwen1.5-0.5B"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Training

In [6]:
from transformers import TrainingArguments,Trainer

batch_size = 1
args = TrainingArguments(
    'outputs', 
    learning_rate=8e-5, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True,
    evaluation_strategy="epoch", 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size * 2,
    num_train_epochs=2, 
    weight_decay=0.01, 
    report_to='none',
)

In [7]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=2048)
    tokenized["labels"] = tokenized["input_ids"][:]
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [8]:
trainer = Trainer(
    model, args, 
    train_dataset=tokenized_dataset['train'], 
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
)

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 